In [1]:
import json
from pyfuseki import FusekiUpdate
import httpx
from pysolr import Solr

In [20]:
upAcervo = FusekiUpdate('http://localhost:3030', 'authority')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upAcervo.run_sparql(d)
response.convert()

solrAuthority = Solr('http://localhost:8983/solr/authority/', timeout=10)
solrAuthority.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">308</int>\n</lst>\n</response>\n'

In [3]:
with open("agents.json", encoding="utf-8") as file:
    reader = file.read()
    agents = json.loads(reader)
    file.close()

agents

[{'type': 'PersonalName',
  'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
   'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
   'creationDate': '2023-08-31',
   'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
   'generationProcess': 'BiblioKeia v.1',
   'generationDate': '2023-08-31T14:59:39',
   'identifiedBy': [{'type': 'Lccn',
     'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
     'value': 'n80002329'}],
   'status': {'value': 'mstatus:new', 'label': 'novo'}},
  'authoritativeLabel': 'Machado de Assis, 1839-1908',
  'elementList': [{'type': 'FullNameElement',
    'elementValue': {'value': 'Machado de Assis,', 'lang': None}},
   {'type': 'DateNameElement',
    'elementValue': {'value': '1839-1908', 'lang': None}}],
  'fullerName': {'type': 'PersonalName',
   'elementValue': {'value': 'Joaquim Maria Machado', 'lang': None}},
  'birthDate': '1839-06-21',
  'birthPlace': 'Rio de Janeiro

In [8]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_data = agents[0]
response = httpx.post('http://localhost:8000/authorities/agents/', headers=headers, json=json_data)
response

<Response [201 Created]>

In [22]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

for agent in agents[1:]:
    response = httpx.post('http://localhost:8000/authorities/agents/', headers=headers, json=agent, timeout=1000)
    print(response.json())


{'id': 'bka-2', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":255\n  }\n}'}
{'id': 'bka-3', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":253\n  }\n}'}
{'id': 'bka-4', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":221\n  }\n}'}
{'id': 'bka-5', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":234\n  }\n}'}
{'id': 'bka-6', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":268\n  }\n}'}


In [2]:
from pydantic import BaseModel
from typing import Optional
from api.src.schemas.authorities.authority import Variant, AdminMetadata, Element

class Uri(BaseModel):
    value: str
    label: str
    base: Optional[str] = None

class Affiliation(BaseModel):
    organization: Uri
    affiliationStart: Optional[str] = None
    affiliationEnd: Optional[str] = None
    
class Agents(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    authoritativeLabel: str
    elementList: list[Element]
    fullerName: Optional[Element] = None
    birthDate: Optional[str] = None
    birthPlace: Optional[str] = None
    deathDate: Optional[str] = None
    hasAffiliation: Optional[list[Affiliation]] = None
    occupation: Optional[list[Uri]] = None
    fieldOfActivity: Optional[list[Uri]] = None
    hasCloseExternalAuthority: Optional[list[Uri]] = None
    hasExactExternalAuthority: Optional[list[Uri]] = None
    hasVariant: Optional[list[Variant]] = None
    subjectOf: Optional[list[Uri]] = None
    contributorOf: Optional[list[Uri]] = None
    isMemberOfMADSCollection: str

In [5]:
agent =agents[0]
request = Agents(**agent)
request

Agents(type='PersonalName', adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/dlc', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 8, 31), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate='2023-08-31T14:59:39', identifiedBy=[IdentifiedBy(type='Lccn', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='n80002329')], status=Status(value='mstatus:new', label='novo')), authoritativeLabel='Machado de Assis, 1839-1908', elementList=[Element(type='FullNameElement', elementValue=Label(value='Machado de Assis,', lang=None)), Element(type='DateNameElement', elementValue=Label(value='1839-1908', lang=None))], fullerName=Element(type='PersonalName', elementValue=Label(value='Joaquim Maria Machado', lang=None)), birthDate='1839-06-21', birthPlace='Rio de Janeiro (Brazil)', deathDate='1908-09-29', hasAffiliation=[Affiliation(org

In [9]:
request.authoritativeLabel

'Machado de Assis, 1839-1908'

In [17]:
from api.src.function.authorities.makeLabel import MakeLabel

def GetImagem(uri):
    id = uri.split('/')[-1]

    url = 'https://www.wikidata.org/w/api.php'
    params = {
                'action': 'wbgetentities',
                'ids': id,
                'props': 'claims',
                'languages': 'pt',
                'format': 'json'
            }
    response = httpx.get(url, params=params) 
    response = response.json()
    if response.get('error'):
        return False
    else:
        # file = response['entities'][id]['claims']['P18'][0]['mainsnak']['datavalue']['value']
        file = response['entities'][id]['claims'].get('P18')
        if file:
            img = file[0]['mainsnak']['datavalue']['value']
            imagem = f'http://commons.wikimedia.org/wiki/Special:FilePath/{img}'
            return imagem
        else:
            return False
        
def MakeDocAgents(request, id):

    doc = { 
            'id': id,
            'type': request.type,
            "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
            # "label": f'{MakeLabel(request.elementList)}' ,
            "authority": request.authoritativeLabel,
            "isMemberOfMADSCollection": request.isMemberOfMADSCollection
        }
    
    if request.fullerName:
        doc['fullerName'] = request.fullerName.elementValue.value
    if request.birthDate:
        doc['birthDate'] = request.birthDate
    if request.birthPlace:
        doc['birthPlace'] = request.birthPlace
    if request.deathDate:
        doc['deathDate'] = request.deathDate
    
    # hasAffiliation  
    if request.hasAffiliation:
        affiliations = list()
        for i in request.hasAffiliation:

            a = {
                'id': f"{id}/hasAffiliation#{i.organization.value.split('/')[-1]}",
                'organization': i.organization.label,
                'affiliationStart': i.affiliationStart,
            }
            if i.affiliationEnd:
                a['affiliationEnd'] = i.affiliationEnd
            affiliations.append(a)
        doc['hasAffiliation'] = affiliations
        doc['affiliation']  = [i['organization'] for i in affiliations]

    
    # hasVariant
    if request.hasVariant:
        variants = list()
        for i in request.hasVariant:
            label = [j.elementValue.value for j in i.elementList]
            label = " ".join(label)
            variants.append(label)
        doc['variant'] = variants

    # hasCloseExternalAuthority
    if request.hasCloseExternalAuthority:
        uris = list()
        for i in request.hasCloseExternalAuthority:
            uri = {
                    'id': f"{id}/hasCloseExternalAuthority#{i.value.split('/')[-1]}",
                    'uri': i.value, 
                    'label': i.label, 
                    'base': i.base }
            uris.append(uri)
            if i.base == 'www.wikidata.org':
                imagem = GetImagem(i.value)
                if imagem:
                    doc['imagem'] = imagem
        doc['hasCloseExternalAuthority'] = uris

    # Occupation
    if request.occupation:
        occupations = list()
        for i in request.occupation:
            uri = {
                    'id': f"{id}/occupation#{i.value.split('/')[-1]}",
                    'uri': i.value, 
                    'label': i.label, 
                    'base': i.base }
            occupations.append(uri)
        doc['hasOccupation'] = occupations
        doc['occupation']  = [i['label'] for i in occupations]


    # fieldOfActivity
    if request.fieldOfActivity:
        fields = list()
        for i in request.fieldOfActivity:
            uri = {
                    'id': f"{id}/fieldOfActivity#{i.value.split('/')[-1]}",
                    'uri': i.value, 
                    'label': i.label, 
                    'base': i.base }
            fields.append(uri)
        doc['fieldOfActivity'] = fields

    return doc

doc = MakeDocAgents(request, "bka-1")
doc


{'id': 'bka-1',
 'type': 'PersonalName',
 'creationDate': '2023-08-31',
 'authority': 'Machado de Assis, 1839-1908',
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/PersonalName/',
 'fullerName': 'Joaquim Maria Machado',
 'birthDate': '1839-06-21',
 'birthPlace': 'Rio de Janeiro (Brazil)',
 'deathDate': '1908-09-29',
 'hasAffiliation': [{'id': 'bka-1/hasAffiliation#n82059239',
   'organization': 'Academia Brasileira de Letras',
   'affiliationStart': '1897',
   'affiliationEnd': '1908'}],
 'affiliation': ['Academia Brasileira de Letras'],
 'variant': ['Assis, Joaquim Maria Machado de, 1839-1908',
  'Assis, Machado de, 1839-1908',
  'De Assis, Joaquim Maria Machado, 1839-1908',
  'De Assis, Machado, 1839-1908',
  'Machado de Assis, Joaquim Maria, 1839-1908',
  'Machado de Assis, Joaquín María, 1839-1908',
  'Mashado de Assiz, Zhoakin, 1839-1908',
  'Semana Dr., 1839-1908'],
 'imagem': 'http://commons.wikimedia.org/wiki/Special:FilePath/Machado de Assis aos 57 anos.jpg',


In [15]:
x = [i['organization'] for i in affiliations]
x

['Academia Brasileira de Letras']